In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

%matplotlib inline
%load_ext autoreload
%autoreload 2
%aimport mwtools
from mwtools import nemo


## The package has a few useful things for querying, cross matching, and playing with MW models

In [ ]:
help(mwtools)

## How to query the WSA (mostly for UKIDSS data) and the VSA (mostly for VVV data)

We use query_vsa and query_wsa to submit SQL queries and get the results.

Note theres also virac in the VSA as vvvProperMotionCatalogue -- for the full list go to http://horus.roe.ac.uk/vsa/www/vsa_browser.html or http://wsa.roe.ac.uk/www/wsa_browser.html)

In [ ]:
help(mwtools.query)

In [ ]:
central_l,central_b = 4.75, 1.25
width_l,width_b = 0.5, 0.1

In [ ]:
sqlquery="""select ra,dec,l,b,japermag3 as J,hapermag3 as H, k_1AperMag3 as K
from gpsJHKsource where l between {} and {} and b between {} and {} 
and japermag3-k_1AperMag3 > 0 and k_1AperMag3 between 5 and 18""".format(central_l-0.5*width_l,central_l+0.5*width_l,
                                                                        central_b-0.5*width_b,central_b+0.5*width_b)
ukidss=mwtools.query_wsa(sqlquery)
ukidss.RA = np.rad2deg(ukidss.RA) #be careful, ra and dec are in radians in the VSA/WSA
ukidss.DEC = np.rad2deg(ukidss.DEC)
ukidss[0:5]

In [ ]:
sqlquery="""select ra,dec,l,b,japermag3 as J,hapermag3 as H, ksAperMag3 as K
from vvvSource where l between {} and {} and b between {} and {} 
and japermag3-ksAperMag3 > 0 and ksAperMag3 between 5 and 18""".format(central_l-0.5*width_l,central_l+0.5*width_l,
                                                                        central_b-0.5*width_b,central_b+0.5*width_b)
vvv=mwtools.query_vsa(sqlquery)
vvv.RA = np.rad2deg(vvv.RA) #be careful, ra and dec are in radians in the VSA/WSA
vvv.DEC = np.rad2deg(vvv.DEC)
vvv[0:5]

The WSA and VSA also have many other tables too... e.g. 2MASS/Gaia but instead here we use vizier. Almost all useful tables are in vizier (at least if they aren't too big like Gaia/UKIDSS/VVV) so it's useful to be able to query directly in python

For this there's astroquery extension to astropy

In [ ]:
from astroquery.vizier import Vizier
import astropy.coordinates as coord
import astropy.units as u

v = Vizier(columns=["*", "+_r"],catalog="II/246")
v.ROW_LIMIT=-1
result = v.query_region(coord.Galactic(l=central_l*u.deg, b=central_b*u.deg),
                        width=width_l*u.deg, height=width_b*u.deg, catalog="II/246")
tmass=result[0].to_pandas()
tmass[0:5]

In [ ]:
fig,axs=plt.subplots(1,3,sharey=True,sharex=True)

axs[0].plot(ukidss.J-ukidss.K,ukidss.K,'.',markersize=0.5)
axs[1].plot(vvv.J-vvv.K,vvv.K,'.',markersize=0.5)
axs[2].plot(tmass.Jmag-tmass.Kmag,tmass.Kmag,'.',markersize=0.5)
axs[0].set_title('UKIDSS')
axs[1].set_title('VVV')
axs[2].set_title('2MASS')

axs[0].set_ylim([16,9])
axs[0].set_xlim([0,3])

axs[1].set_xlabel('J-K')
axs[0].set_ylabel('K')
plt.subplots_adjust(hspace = 0,wspace = 0.1)

In [ ]:
fig,ax = plt.subplots(1,1)

bins = np.arange(6,18,0.2)
ax.hist(ukidss.K,bins,label='UKIDSS',alpha=0.3,histtype='step')
ax.hist(vvv.K,bins,label='VVV',alpha=0.3,histtype='step')
ax.hist(tmass.Kmag,bins,label='2MASS',alpha=0.3,histtype='step')
qflg=tmass.Qflg.str.decode("utf-8")
cut=(tmass.e_Jmag < 0.1) & (tmass.e_Hmag < 0.1) & (tmass.e_Kmag < 0.1) & \
    ((qflg.str[0] == 'A') | (qflg.str[0] == 'B')) & ((qflg.str[1] == 'A') | (qflg.str[1] == 'B')) & \
    ((qflg.str[2] == 'A') | (qflg.str[2] == 'B')) & (tmass.Cflg == b'000')
ax.hist(tmass[cut].Kmag,bins,label='2MASS Cut',alpha=0.3,histtype='step')

ax.legend()
ax.set_yscale('log')
ax.set_xlabel('K')
ax.set_ylabel('Number of Stars')
ax.set_title(r'Region centered on $(l,b)={:.2f},{:.2f}$'.format(central_l,central_b))



## One option for cross matching to Gaia

There's a few different ways to cross match to gaia
- you could download the entire source table and cross match locally, but this is 550 GB, and doing it efficiently isn't trivial
- for smaller tables the CDS xmatch service is fast, but doesn't give all columns of Gaia DR2 e.g. proper motion errors
- WSA/VSA should already cross matches to UKIDSS/VVV already (I think)
- you could use the gaia archive

Here we take the last approach, but do it programatically in python, rather than logging into the website. There will be limits to the size of the cross matches, but I've tried with 200,000 sources I didn't reach them yet.

In [ ]:
help(mwtools.Gaia_DR2_Xmatch)

In [ ]:
vvv_x_gaiadr2 = mwtools.Gaia_DR2_Xmatch(vvv,dist=1)
vvv_x_gaiadr2[0:5]

The column dist is the distance to the nearest cross match (actually I need to check if it's the neasest...)

If we look at the distribution we see that we probably shouldn't trust cross matches beyond 0.4"

In [ ]:
plt.hist(vvv_x_gaiadr2[np.isfinite(vvv_x_gaiadr2.dist)].dist*3600,100)
print('Fraction cross matched to <0.4" : {:.2f}'.format(np.sum(vvv_x_gaiadr2.dist < 0.4)/len(vvv)))
plt.yscale('log')
plt.xlabel('Distance between VVV DR4 and Gaia DR2 sources [arcsec]')
_=plt.ylabel('Number of stars')

Gaia gives proper motions and proper motion errors/covariances in equitorial coordinates (ra,dec). We generally prefer galactic coordinates. The function add_gaia_galactic_pms adds them. 

In [ ]:
mwtools.add_gaia_galactic_pms(vvv_x_gaiadr2)

And the errors are fairly small...

In [ ]:
goodxmatch = (vvv_x_gaiadr2.dist < 0.4) & np.isfinite(vvv_x_gaiadr2['pml'])

f, (ax_pm, ax_pm_err) = plt.subplots(nrows=1,ncols=2,sharey=True)

bins=np.arange(-25,25,1)
_ = ax_pm.hist(vvv_x_gaiadr2[goodxmatch]['pml'],bins,alpha=0.3,label=r'$\mu_l$')
_ = ax_pm.hist(vvv_x_gaiadr2[goodxmatch]['pmb'],bins,alpha=0.3,label=r'$\mu_b$')
_ = ax_pm.set_xlabel('mas/yr')
_ = ax_pm.legend()
bins=np.arange(0,3,0.02)
_ = ax_pm_err.hist(vvv_x_gaiadr2[goodxmatch]['pml_error'],bins,alpha=0.3,label=r'Error in $\mu_l$')
_ = ax_pm_err.hist(vvv_x_gaiadr2[goodxmatch]['pmb_error'],bins,alpha=0.3,label=r'Error in $\mu_b$')
_ = ax_pm_err.set_xlabel('mas/yr')
_ = ax_pm_err.legend()
_ = ax_pm.set_ylabel('Number of stars')
plt.subplots_adjust(hspace = 0,wspace = 0)

But in this field, although we have a good number of cross matches fairly deep, to K~14.5, they are typically faint already at 12 to have accurate proper motions

In [ ]:
goodxmatch = (vvv_x_gaiadr2.dist < 0.4)
goodpm = (vvv_x_gaiadr2.dist < 0.4) & (np.sqrt(vvv_x_gaiadr2.pml_error**2 + vvv_x_gaiadr2.pmb_error**2) < 1.5)

bins=np.arange(10,16,0.25)
H,edges=np.histogram(vvv_x_gaiadr2.K,bins)
Hgood,edges=np.histogram(vvv_x_gaiadr2[goodxmatch].K,bins)
Hgoodpm,edges=np.histogram(vvv_x_gaiadr2[goodpm].K,bins)

centers=0.5*(edges[:-1]+edges[1:])
plt.plot(centers,Hgood/H,label='Gaia DR2 Cross matches <0.4"')
plt.plot(centers,Hgoodpm/H,label='Gaia DR2 Proper motion error < 1.5mas/yr')
plt.legend()
plt.ylabel('Fraction of VVV sources')
plt.xlabel('$K_s$ [mag]')

## An NMagicParticles class

I have a class that might be useful for loading and playing with nmagic models in python:

In [ ]:
help(mwtools.NMagicParticles)

We can load our model directly from a parameter file. 

If you don't have a model at hand then you can get the Portail et al 2017 model using:

scp nmagic2:/home/wegg/BestModel/{ModelParameters,}M85MW_1100_42.5_1.5_0060000 .

In [ ]:
model=mwtools.NMagicParticles(parameterfile='/Users/wegg/BestModel/ModelParametersM85MW_1000_40.0_2.0_0060000')

Then we can easily do things like look at the velocity distribution in baades window

In [ ]:
stars=model.stellar
r,l,b = stars.rlb.T
baade = (l > -0.5) & (l < 0.5) & (b > -5) & (b < -4)
vr,mul,mub = stars[baade].vrmulmub.T

And see what it looks like

In [ ]:
r=r[baade]
fig,axs = plt.subplots(3,1,figsize=(6,7),sharex=True)
axs[0].plot(r,mul,'.')
axs[0].set_ylabel('$\mu_l$ [mas/yr]')
axs[1].plot(r,mub,'.')
axs[1].set_ylabel('$\mu_b$ [mas/yr]')
axs[2].plot(r,vr,'.')
axs[2].set_ylabel('$v_r$ [km/s]')
axs[2].set_xlabel('Distance [kpc]')

axs[0].set_xlim([0,15])
fig.subplots_adjust(hspace = 0,wspace = 0)

## Calling nemo to compute the potential/circular velocity

#### The nemo submodule has functions to call nemo from python:

In [ ]:
help(nemo)

#### First let's compute the model rotation curve

In [ ]:
help(nemo.rotationcurve)
r, vcirc2_stars = nemo.rotationcurve(model.stellar)
r, vcirc2_dm = nemo.rotationcurve(model.dm)

#### Does the rotation curve look reasonable?

In [ ]:
plt.plot(r,np.sqrt(vcirc2_stars),label='Stellar')
plt.plot(r,np.sqrt(vcirc2_dm),label='DM')
vc_total=np.sqrt(vcirc2_stars+vcirc2_dm)
plt.plot(r,vc_total,label='Total')
plt.legend()
plt.xlabel('$r$ [kpc]')
plt.ylabel('$V_c$ [km/s]')
Vc0=np.interp(model.R0,r,vc_total)
_=plt.title('$V_c(R_0)={:.1f}$ km/s'.format(Vc0))

#### The circular velocity seems higher at the Solar radius compared to the fitted 238km/s. Let's look at it azimuthally. For this we can use getgravity which returns the acceleration and potential at the requested positions:

In [ ]:
help(nemo.getgravity)

#### Whether through luck or judgement, we see that at the solar position (azimuth=0) we are actually much closer to 238 km/s

In [ ]:
ntheta=360
theta = np.linspace(0, 2 * np.pi, num=ntheta, endpoint=False)
x = model.R0 * np.sin(theta)
y = model.R0 * np.cos(theta)
positions = np.zeros((ntheta, 3))
positions[:, 0] = x.flatten()
positions[:, 1] = y.flatten()

ax, ay, az, pot = nemo.getgravity(model, positions).T
ar = - (x * ax + y * ay) / np.sqrt(x * x + y * y)
vc = np.sqrt(ar*model.R0)
plt.plot(theta,vc,label='Raw')

#The particle noise is considerable compared to the size of the variation in Vc so smooth
from scipy import signal
plt.plot(theta, signal.savgol_filter(vc,31,1,mode='mirror'), label='Smoothed')
plt.legend()
plt.xlabel('Azimuth [radians]')
plt.ylabel('$V_c$ [km/s]')

#### We could also use getgravity to compute the potential too. Instead lets use the more convenient gravity_cartesian_grid to do so. There's also gravity_spherical_grid depending on your preference

In [ ]:
help(nemo.gravity_cartesian_grid)

In [ ]:
x = np.linspace(-10,10,50)
y = np.linspace(-10,10,100)
grav = nemo.gravity_cartesian_grid(model, x, y, z=0)

#### Lets compute and plot the effective potential

In [ ]:
grav['omega']=40.
grav['xmat'], grav['ymat'] = np.meshgrid(x, y, indexing='xy')
grav['effpot'] = grav['pot'] - 0.5*(grav['xmat']**2 + grav['ymat']**2)*grav['omega']**2
plt.contourf(grav['xmat'], grav['ymat'], grav['effpot'],25)
plt.gca().set_aspect('equal', adjustable='box')
plt.xlabel('x [kpc]')
plt.ylabel('y [kpc]')
cbar = plt.colorbar()
cbar.set_label('Effective potential [(km/s)$^2$]')

#### We can compare this to the potential computed in nmagic. You can get this using:
#### scp -r nmagic2:/home/wegg/ForOrtwin/nmagicPotential .
#### then inside the folder Output/ there is phir0000000 and phixy0000000 for the potential in polar and cartesian coordiantes respectively
#### or just get these directly scp -r nmagic2:/home/wegg/ForOrtwin/nmagicPotential/Output/phi* .

In [ ]:
def read_nmagic_potential(filename,Viu=1.0):
    """Function to read output from nmagicPotential and convert to physical units if required. 
    **** Function not tested on nx!=ny ***"""
    
    # x and y are flipped due to ij vs xy indexing in nmagic output vs nemo version 
    y,x,phi,phieff =  np.loadtxt(filename, skiprows=1, unpack=True)
    # First line specifies grid, use this to reshape the arrays
    with open(filename) as f:
        firstline = f.readline()
    ny, nx = firstline.split() 
    nx, ny = int(nx), int(ny)
    grav={}
    grav['xmat']=x.reshape((ny,nx)).T
    grav['x']=grav['xmat'][:,0]
    grav['ymat']=y.reshape((ny,nx)).T
    grav['y']=grav['ymat'][0,:]
    grav['pot']=phi.reshape((ny,nx)).T*Viu**2
    grav['effpot']=phieff.reshape((ny,nx)).T*Viu**2
    return grav

nmagic_grav = read_nmagic_potential('phixy0000000',Viu=model.Viu)

In [ ]:
plt.contourf(nmagic_grav['x'], nmagic_grav['y'], nmagic_grav['effpot'],25)
plt.gca().set_aspect('equal', adjustable='box')
plt.xlabel('x [kpc]')
plt.ylabel('y [kpc]')
cbar = plt.colorbar()
cbar.set_label('Effective potential [(km/s)$^2$]')

#### The potentials look qualitatively similar. To quantitatively compare to nemo lets compute on the same grid:

In [ ]:
nemo_grav = nemo.gravity_cartesian_grid(model, nmagic_grav['x'], nmagic_grav['y'], z=0)
nemo_grav['omega']=40.
nemo_grav['xmat'], nemo_grav['ymat'] = np.meshgrid(nemo_grav['x'], nemo_grav['y'], indexing='xy')
nemo_grav['effpot'] = nemo_grav['pot'] - 0.5*(nemo_grav['xmat']**2 + nemo_grav['ymat']**2)*nemo_grav['omega']**2

#### If we look at a slice we see that there's actually a slight offset. This isn't bad, remembering your high school Newtonian physics then there is an ambiguity in the zero point of potentials since none of the forces depend on them.

In [ ]:
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
fig, ax = plt.subplots(1,1)

ax.plot(nmagic_grav['x'],nmagic_grav['effpot'][:,25],label='Nmagic')
ax.plot(nemo_grav['x'],nemo_grav['effpot'][:,25],label='Nemo')
ax.legend()
ax.set_ylabel('Effective potential [(km/s)$^2$]')

offset = np.mean(nmagic_grav['pot'] - nemo_grav['pot'])
print('Overall Offset: {} (km/2)**2'.format(offset))

ax_divider = make_axes_locatable(ax)
ax2 = ax_divider.append_axes("bottom", size="20%", pad="2%")
ax2.plot(nmagic_grav['x'],nmagic_grav['effpot'][:,25]-nemo_grav['effpot'][:,25])
ax2.set_xlabel('x [kpc]')
ax2.set_ylabel('Nmagic $-$\n Nemo')
ax2.set_ylim([0,2*offset])

nmagic_grav['effpot']-=offset # Arbitrarily adjust nmagic potential to nemo convention
nmagic_grav['pot']-=offset

#### And finally we can look at the difference in potential. Theres a ring at 10kpc, presumably caused by the switch in the nmagic potential solver, but the differences are very small

In [ ]:
CS=plt.contourf(nmagic_grav['x'], nmagic_grav['y'], nmagic_grav['effpot'] - nemo_grav['effpot'],
                levels=np.arange(-250,250,50),cmap=cm.coolwarm,vmin=-250,vmax=250)
plt.gca().set_aspect('equal', adjustable='box')
plt.xlabel('x [kpc]')
plt.ylabel('y [kpc]')
cbar = plt.colorbar(CS)
cbar.set_label('Effective potential [(km/s)$^2$]')
